# Code for TSP solving with QAOA

In [ ]:
import os
import random
import time
from collections import deque
from itertools import permutations
import operator

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib
matplotlib.use("TkAgg")  # Set non-interactive backend
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.optimize import minimize

import unittest

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import QAOAAnsatz, TwoLocal
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

import qiskit_aer as Aer
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error

from qiskit.primitives import Sampler, Estimator, StatevectorSampler

from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    SamplerV2,
    SamplerV2 as Sampler2,  # optional, remove if not needed
    Session,
    EstimatorV2 as Estimator3,  # optional, rename or remove as needed
)

from qiskit_algorithms import (
    QAOA,
    VQE,
    SamplingVQE,
    NumPyMinimumEigensolver,
)
from qiskit_algorithms.optimizers import (
    COBYLA,
    SPSA,
    ADAM,
)
from qiskit_algorithms.utils import algorithm_globals

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import Tsp
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import (
    QuadraticProgramToQubo,
    InequalityToEquality,
)

from docplex.mp.model import Model

## Define Problem

In [ ]:
class TSP_Problem:
    def __init__(self, num_cities):
        self.n = num_cities
        """self.tsp = Tsp.create_random_instance(self.n, seed=123)"""
        self.tsp = Tsp.create_random_instance(self.n)
        self.adj_matrix = nx.to_numpy_array(self.tsp.graph)

        self.convert()

    def convert(self):
        self.qp = self.tsp.to_quadratic_program()
        conv = QuadraticProgramToQubo()
        self.qubo = conv.convert(self.qp)
        self.qubitOp, self.offset = self.qubo.to_ising()

    def draw_graph(self, G, colors, pos):
        plt.figure(figsize=(10, 10))
        ax = plt.axes(frameon=True)

        # Draw edges
        nx.draw_networkx_edges(G, pos, width=2, alpha=0.5)

        # Draw nodes
        nx.draw_networkx_nodes(G, pos, node_color=colors, node_size=800, alpha=0.8)

        # Draw node labels
        nx.draw_networkx_labels(G, pos, font_size=14, font_weight="bold")

        # Draw edge labels
        edge_labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=12)

        # Remove axis
        plt.axis("off")

        # Add some padding around the graph
        plt.tight_layout()

## Solving the problem

In [ ]:
def build_mixer_hamiltonian_WH(n_cities):
    """
    Builds the mixer Hamiltonian for TSP that preserves the one-hot encoding constraints.
    For each row and column, creates terms that swap between valid configurations.

    Args:
        n_cities (int): Number of cities in TSP problem

    Returns:
        SparsePauliOp: The mixer Hamiltonian operator
    """
    n = n_cities
    mixer_terms = []
    coeffs = []

    # For each position (row)
    for i in range(n):
        row_start = i * n
        # For each pair of cities in this position
        for j in range(n):
            for k in range(j + 1, n):
                # Create SWAP mixer between cities j and k in position i
                pauli_str = ["I"] * (n * n)
                pauli_str[row_start + j] = "X"
                pauli_str[row_start + k] = "X"
                mixer_terms.append("".join(pauli_str))
                coeffs.append(1.0)

                pauli_str[row_start + j] = "Y"
                pauli_str[row_start + k] = "Y"
                mixer_terms.append("".join(pauli_str))
                coeffs.append(1.0)

    # For each city (column)
    for j in range(n):
        # For each pair of positions for this city
        for i in range(n):
            for k in range(i + 1, n):
                # Create SWAP mixer between positions i and k for city j
                pauli_str = ["I"] * (n * n)
                pauli_str[i * n + j] = "X"
                pauli_str[k * n + j] = "X"
                mixer_terms.append("".join(pauli_str))
                coeffs.append(1.0)

                pauli_str[i * n + j] = "Y"
                pauli_str[k * n + j] = "Y"
                mixer_terms.append("".join(pauli_str))
                coeffs.append(1.0)

    return SparsePauliOp(mixer_terms, coeffs)


def build_mixer_hamiltonian_RS(n_cities):
    """
    Builds the Random Swap (RS) mixer Hamiltonian for TSP.
    HMRS = sum_{i=0}^{n-2} sum_{j=i+1}^{n-1} prod_{t=0}^{n-1} XY_{(i,t),(j,t)}

    Args:
        n_cities (int): Number of cities in TSP problem

    Returns:
        SparsePauliOp: The RS mixer Hamiltonian operator
    """
    n = n_cities
    mixer_terms = []
    coeffs = []

    # For each pair of cities (i,j)
    for i in range(n - 1):
        for j in range(i + 1, n):
            # For each position t
            pauli_str = ["I"] * (n * n)

            # Apply XY terms for each position
            for t in range(n):
                idx1 = t * n + i  # Index for city i at position t
                idx2 = t * n + j  # Index for city j at position t

                # Create XY term
                pauli_str[idx1] = "X"
                pauli_str[idx2] = "Y"
                mixer_terms.append("".join(pauli_str))
                coeffs.append(1.0)

                # Create YX term (conjugate)
                pauli_str[idx1] = "Y"
                pauli_str[idx2] = "X"
                mixer_terms.append("".join(pauli_str))
                coeffs.append(-1.0)  # Note the minus sign for the conjugate term

                # Reset for next position
                pauli_str[idx1] = "I"
                pauli_str[idx2] = "I"

    return SparsePauliOp(mixer_terms, coeffs)


def build_walsh_hadamard_state(n_cities):
    """
    Build the Walsh-Hadamard state |sw_m⟩ = (Wn|0⟩)^⊗n where
    Wn = 1/√N (∑|i⟩) for i=0 to N-1

    Args:
        n_cities (int): Number of cities in TSP problem

    Returns:
        QuantumCircuit: Circuit implementing the Walsh-Hadamard state
    """
    # We need n_cities * n_cities qubits since each city position
    # needs n_cities qubits to encode
    n_qubits = n_cities * n_cities

    # Create quantum registers and circuit
    qr = QuantumRegister(n_qubits, "q")
    circ = QuantumCircuit(qr)

    # Apply Hadamard gates to create uniform superposition
    # For each city position
    for i in range(n_cities):
        # Apply H gates to the qubits encoding this position
        for j in range(n_cities):
            qubit_idx = i * n_cities + j
            circ.h(qr[qubit_idx])

    return circ

In [ ]:
def get_edges(cycle):
    edges1 = {frozenset({cycle[i], cycle[i + 1]}) for i in range(len(cycle) - 1)}
    return edges1


def are_cycles_equal(cycle1, cycle2):
    edges1 = get_edges(cycle1)
    edges2 = get_edges(cycle2)
    return edges1 == edges2

In [ ]:
class OptCostSorted:
    def __init__(self, problem, reps, shots, tol=1e-4, mixer=None, init_state=None):
        self.problem = problem
        self.reps = reps
        self.shots = shots
        self.qubo = problem.qubo
        self.qubitOp = problem.qubitOp
        self.offset = problem.offset
        self.n = problem.n
        self.adj_matrix = problem.adj_matrix
        self.tsp = problem.tsp
        self.init_state = init_state
        self.qp = problem.qp
        self.results = []
        self.optimal_cost = 0
        self.optimal_order = []
        self.objective_func_vals = []
        self.tol = tol
        self.mixer = mixer
        self.backend = AerSimulator(device="GPU")
        print(self.backend)
        self.best_cost_brute, self.best_order_brute = self.brute_force_tsp()
        print(self.best_order_brute)
        self.build_ini_circ()
        self.transpile_circ()
        self.initialize_circ_params()
        """self.get_warm_start_params()"""

    def brute_force_tsp(self):
        w = self.adj_matrix
        N = self.n
        a = list(permutations(range(1, N)))
        last_best_distance = 1e10
        for i in a:
            distance = 0
            pre_j = 0
            for j in i:
                distance = distance + w[j, pre_j]
                pre_j = j
            distance = distance + w[pre_j, 0]
            order = (0,) + i
            if distance < last_best_distance:
                best_order = order
                last_best_distance = distance
        order = [i for i in best_order]
        order.append(order[0])
        return last_best_distance, order

    def solve(self):
        self.optimize_circ_params()
        self.run_optimized_circuit()
        worked = self.find_optimal_cost_order()
        if worked:
            return 1, self.best_order_qaoa, self.best_cost_brute, self.time
        else:
            return 0, self.best_order_qaoa, self.best_cost_brute, self.time

    def build_ini_circ(self):
        reps = self.reps
        if self.init_state is not None:
            self.init_state = build_walsh_hadamard_state(self.n)
        if self.mixer is None:
            self.qubo_circuit = QAOAAnsatz(
                cost_operator=self.qubitOp,
                reps=reps,
                initial_state=self.init_state,
            )
        elif self.mixer == "RS":
            self.qubo_circuit = QAOAAnsatz(
                cost_operator=self.qubitOp,
                reps=reps,
                mixer_operator=build_mixer_hamiltonian_RS(self.n),
                initial_state=self.init_state,
            )
        elif self.mixer == "WH":
            self.qubo_circuit = QAOAAnsatz(
                cost_operator=self.qubitOp,
                reps=reps,
                mixer_operator=build_mixer_hamiltonian_WH(self.n),
                initial_state=self.init_state,
            )
        self.qubo_circuit.measure_all()

    def transpile_circ(self):
        pm = generate_preset_pass_manager(optimization_level=3, backend=self.backend)
        self.candidate_circuit = pm.run(self.qubo_circuit)

    def initialize_circ_params(self, reps=0):
        if reps == 0:
            reps = self.reps
        initial_gamma = np.pi
        initial_beta = np.pi / 2
        self.init_params = []
        for i in range(reps):
            initial_gamma = 2 * np.pi * random.random()
            initial_beta = np.pi * random.random()
            self.init_params.append(initial_gamma)
            self.init_params.append(initial_beta)

    def cost_func_estimator(self, params, ansatz, hamiltonian, estimator):

        # transform the observable defined on virtual qubits to
        # an observable defined on all physical qubits
        isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

        pub = (ansatz, isa_hamiltonian, params)
        job = estimator.run([pub])

        results = job.result()[0]
        cost = results.data.evs
        total_cost = cost + self.offset

        self.objective_func_vals.append(total_cost)

        return total_cost

    def get_warm_start_params(self):
        """Generate warm start parameters using classical approximation"""
        # Use nearest neighbor solution as initial guess
        nn_solution = self.nearest_neighbor_solution()

        # Ensure we don't take sqrt of negative number or divide by zero
        ratio = max(0.0, min(1.0, nn_solution.cost / (self.best_cost_brute + 1e-10)))
        initial_gamma = np.arcsin(np.sqrt(ratio))
        initial_beta = np.pi / 4  # Start in equal superposition

        self.init_params = []
        for i in range(self.reps):
            factor = max(0.1, 1 - i / self.reps)  # Prevent params from going to zero
            self.init_params.extend([initial_gamma * factor, initial_beta * factor])

    def nearest_neighbor_solution(self):
        """
        Implements nearest neighbor algorithm for TSP
        Returns a solution object with path and cost
        """

        class Solution:
            def __init__(self, path, cost):
                self.path = path
                self.cost = cost

        n = self.n
        w = self.adj_matrix

        # Start from node 0
        current = 0
        path = [current]
        unvisited = set(range(1, n))
        total_cost = 0

        # Build path
        while unvisited:
            next_node = min(unvisited, key=lambda x: w[current, x])
            total_cost += w[current, next_node]
            current = next_node
            path.append(current)
            unvisited.remove(current)

        # Return to start
        total_cost += w[current, 0]
        path.append(0)

        return Solution(path, total_cost)

    def optimize_circ_params(self):
        """for i in range(self.reps):"""

        with Session(backend=self.backend) as session:
            # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
            estimator = Estimator(mode=session)
            estimator.options.default_shots = self.shots
            start_time = time.time()
            self.result = minimize(
                self.cost_func_estimator,
                self.init_params,
                args=(self.candidate_circuit, self.qubitOp, estimator),
                method="COBYLA",
                tol=self.tol,
            )
            end_time = time.time()
            self.time = end_time - start_time
        best_sol = self.result.x
        best_cost = self.result.fun

        for i in range(5):
            self.init_params = self.result.x
            with Session(backend=self.backend) as session:
                estimator = Estimator(mode=session)
                estimator.options.default_shots = self.shots
                start_time = time.time()
                self.result = minimize(
                    self.cost_func_estimator,
                    self.init_params,
                    args=(self.candidate_circuit, self.qubitOp, estimator),
                    method="COBYLA",
                    tol=self.tol,
                )
            if self.result.fun < best_cost:
                best_cost = self.result.fun
                best_sol = self.result.x
        self.result.x = best_sol

    def run_optimized_circuit(self):
        print(self.result.x)
        self.optimized_circuit = self.candidate_circuit.assign_parameters(self.result.x)
        sampler = Sampler(mode=self.backend)
        sampler.options.default_shots = self.shots

        pub = (self.optimized_circuit,)
        job = sampler.run([pub], shots=self.shots)

        counts_int = job.result()[0].data.meas.get_int_counts()
        counts_bin = job.result()[0].data.meas.get_counts()
        shots = sum(counts_int.values())
        self.final_distribution_bin = {
            key: val / shots for key, val in counts_bin.items()
        }

    def find_optimal_cost_order(self):
        # Sort both distributions by probability in descending order
        sorted_dist_bin = dict(
            sorted(
                self.final_distribution_bin.items(), key=lambda x: x[1], reverse=True
            )
        )

        # Print top 10 most likely outcomes
        top_bin_list = []
        for k, v in list(sorted_dist_bin.items())[:50]:
            temp_list = list(k)
            top_bin_list.append([int(re) for re in temp_list])

        # Example usage
        cost_l = []
        for bin in top_bin_list:
            cost = self.calculate_cost_from_qubo(bin)
            cost_l.append(cost)

        min_index, min_value = min(enumerate(cost_l), key=operator.itemgetter(1))
        most_likely_bitstring = top_bin_list[min_index]
        print(most_likely_bitstring)
        self.best_order_qaoa = []
        for i in range(self.n):
            pos = most_likely_bitstring[self.n * i : self.n * (i + 1)]

            if 1 in pos:
                self.best_order_qaoa.append(self.n - pos.index(1) - 1)
            else:
                return False
        self.best_order_qaoa.append(self.best_order_qaoa[0])
        correct = are_cycles_equal(self.best_order_qaoa, self.best_order_brute)
        return correct

    def calculate_cost_from_qubo(self, bitstring):
        # Convert bitstring to a dictionary of variable assignments
        var_assignment = {
            var.name: bit for var, bit in zip(self.qubo.variables, bitstring)
        }

        # Evaluate the cost function
        cost = self.qubo.objective.evaluate(var_assignment)

        return cost